This notebook is mostly coming from fastai nbs. 

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os
import re

In [3]:
#Uploading our data to vars
train = pd.read_csv('../rossmann-store-sales/train.csv')
store = pd.read_csv('../rossmann-store-sales/store.csv')
test = pd.read_csv('../rossmann-store-sales/test.csv')
sample = pd.read_csv('../rossmann-store-sales/sample_submission.csv')

googletrend = pd.read_csv('../rossmann_additional/googletrend.csv')
state_names = pd.read_csv('../rossmann_additional/state_names.csv')
store_states = pd.read_csv('../rossmann_additional/store_states.csv')
weather = pd.read_csv('../rossmann_additional/weather.csv')

/home/f/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
train.Date.min(), train.Date.max()

('2013-01-01', '2015-07-31')

In [7]:
test.Date.min(), test.Date.max()

('2015-08-01', '2015-09-17')

In [40]:
#function to display a dataframe up to 200 rows and columns
def display_all(df):
    with pd.option_context("display.max_rows", 200, "display.max_columns", 200): 
        display(df)

In [41]:
train

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


In [42]:
test

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0
...,...,...,...,...,...,...,...,...
41083,41084,1111,6,2015-08-01,1.0,0,0,0
41084,41085,1112,6,2015-08-01,1.0,0,0,0
41085,41086,1113,6,2015-08-01,1.0,0,0,0
41086,41087,1114,6,2015-08-01,1.0,0,0,0


In [43]:
store

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1112,1113,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1113,1114,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


In [44]:
sample

,Id,Sales
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
41083,41084,0
41084,41085,0
41085,41086,0
41086,41087,0


In [45]:
googletrend

,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67
...,...,...,...
2067,Rossmann_DE_SL,2015-08-30 - 2015-09-05,95
2068,Rossmann_DE_SL,2015-09-06 - 2015-09-12,47
2069,Rossmann_DE_SL,2015-09-13 - 2015-09-19,80
2070,Rossmann_DE_SL,2015-09-20 - 2015-09-26,57


In [46]:
state_names

,StateName,State
0,BadenWuerttemberg,BW
1,Bayern,BY
2,Berlin,BE
3,Brandenburg,BB
4,Bremen,HB
5,Hamburg,HH
6,Hessen,HE
7,MecklenburgVorpommern,MV
8,Niedersachsen,"HB,NI"
9,NordrheinWestfalen,NW


In [47]:
store_states

,Store,State
0,1,HE
1,2,TH
2,3,NW
3,4,BE
4,5,SN
...,...,...
1110,1111,NW
1111,1112,NW
1112,1113,SH
1113,1114,HH


In [48]:
display_all(weather.T)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,...,15740,15741,15742,15743,15744,15745,15746,15747,15748,15749,15750,15751,15752,15753,15754,15755,15756,15757,15758,15759,15760,15761,15762,15763,15764,15765,15766,15767,15768,15769,15770,15771,15772,15773,15774,15775,15776,15777,15778,15779,15780,15781,15782,15783,15784,15785,15786,15787,15788,15789,15790,15791,15792,15793,15794,15795,15796,15797,15798,15799,15800,15801,15802,15803,15804,15805,15806,15807,15808,15809,15810,15811,15812,15813,15814,15815,15816,15817,15818,15819,15820,15821,15822,15823,15824,15825,15826,15827,15828,15829,15830,15831,15832,15833,15834,15835,15836,15837,15838,15839
file,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,NordrheinWestfalen,...,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland,Saarland
Date,2013-01-01,2013-01-02,2013-01-03,2013-01-04,2013-01-05,2013-01-06,2013-01-07,2013-01-08,2013-01-09,2013-01-10,2013-01-11,2013-01-12,2013-01-13,2013-01-14,2013-01-15,2013-01-16,2013-01-17,2013-01-18,2013-01-19,2013-01-20,2013-01-21,2013-01-22,2013-01-23,2013-01-24,2013-01-25,2013-01-26,2013-

In [49]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, suffixes=("", suffix))

In [50]:
weather = join_df(weather, state_names, 'file', 'StateName')

In [51]:
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]
googletrend['State'] = googletrend.file.str.split('_', expand=True)[2]
googletrend.loc[googletrend.State=='NI', "State"] = 'HB,NI'

In [52]:
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [53]:
#adding date related features to these dataframes by using pandas datetime attributes:
add_datepart(weather, "Date", drop=False)
add_datepart(googletrend, "Date", drop=False)
add_datepart(train, "Date", drop=False)
add_datepart(test, "Date", drop=False)

In [54]:
trend_de = googletrend[googletrend.file == 'Rossmann_DE']

In [55]:
store = join_df(store, store_states, "Store")
len(store[store.State.isnull()])

0

In [56]:
joined = join_df(train, store, "Store")
joined_test = join_df(test, store, "Store")
len(joined[joined.StoreType.isnull()]),len(joined_test[joined_test.StoreType.isnull()])

(0, 0)

In [57]:
joined.transpose()

,0,1,2,3,4,5,6,7,8,9,...,1017199,1017200,1017201,1017202,1017203,1017204,1017205,1017206,1017207,1017208
Store,1,2,3,4,5,6,7,8,9,10,...,1106,1107,1108,1109,1110,1111,1112,1113,1114,1115
DayOfWeek,5,5,5,5,5,5,5,5,5,5,...,2,2,2,2,2,2,2,2,2,2
Date,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,...,2013-01-01 00:00:00,2013-01-01 00:00:00,2013-01-01 00:00:00,2013-01-01 00:00:00,2013-01-01 00:00:00,2013-01-01 00:00:00,2013-01-01 00:00:00,2013-01-01 00:00:00,2013-01-01 00:00:00,2013-01-01 00:00:00
Sales,5263,6064,8314,13995,4822,5651,15344,8492,8565,7185,...,0,0,0,0,0,0,0,0,0,0
Customers,555,625,821,1498,559,589,1414,833,687,681,...,0,0,0,0,0,0,0,0,0,0
Open,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Promo,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
StateHoliday,0,0,0,0,0,0,0,0,0,0,...,a,a,a,a,a,a,a,a,a,a
SchoolHoliday,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Year,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,...,2013,2013,2013,2013,2013,2013,2013,2013,2013,2013


In [58]:
joined = join_df(joined, googletrend, ["State","Year", "Week"])
joined_test = join_df(joined_test, googletrend, ["State","Year", "Week"])
len(joined[joined.trend.isnull()]),len(joined_test[joined_test.trend.isnull()])

(0, 0)

In [59]:
joined = joined.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
joined_test = joined_test.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
len(joined[joined.trend_DE.isnull()]),len(joined_test[joined_test.trend_DE.isnull()])

(0, 0)

In [60]:
joined = join_df(joined, weather, ["State","Date"])
joined_test = join_df(joined_test, weather, ["State","Date"])
len(joined[joined.Mean_TemperatureC.isnull()]),len(joined_test[joined_test.Mean_TemperatureC.isnull()])

(0, 0)

In [61]:
joined.columns, len(joined.columns)

(Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
        'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week', 'Day',
        'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
        'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start',
        'Elapsed', 'StoreType', 'Assortment', 'CompetitionDistance',
        'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
        'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'State', 'file',
        'week', 'trend', 'Date_y', 'Month_y', 'Day_y', 'Dayofweek_y',
        'Dayofyear_y', 'Is_month_end_y', 'Is_month_start_y', 'Is_quarter_end_y',
        'Is_quarter_start_y', 'Is_year_end_y', 'Is_year_start_y', 'Elapsed_y',
        'file_DE', 'week_DE', 'trend_DE', 'Date_DE', 'State_DE', 'Month_DE',
        'Day_DE', 'Dayofweek_DE', 'Dayofyear_DE', 'Is_month_end_DE',
        'Is_month_start_DE', 'Is_quarter_end_DE', 'Is_quarter_start_DE',
        'Is_year_end_DE', 'Is_y

In [62]:
#drop the repetitive columns that hes come from weather and googletrend.
for df in (joined, joined_test):
    for c in df.columns:
        if c.endswith('_y'):
            if c in df.columns: df.drop(c, inplace=True, axis=1)

In [63]:
joined.columns, len(joined.columns)

(Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
        'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week', 'Day',
        'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
        'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start',
        'Elapsed', 'StoreType', 'Assortment', 'CompetitionDistance',
        'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
        'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'State', 'file',
        'week', 'trend', 'file_DE', 'week_DE', 'trend_DE', 'Date_DE',
        'State_DE', 'Month_DE', 'Day_DE', 'Dayofweek_DE', 'Dayofyear_DE',
        'Is_month_end_DE', 'Is_month_start_DE', 'Is_quarter_end_DE',
        'Is_quarter_start_DE', 'Is_year_end_DE', 'Is_year_start_DE',
        'Elapsed_DE', 'Max_TemperatureC', 'Mean_TemperatureC',
        'Min_TemperatureC', 'Dew_PointC', 'MeanDew_PointC', 'Min_DewpointC',
        'Max_Humidity', 'Mean_Humidity', 'Min_Humidity',
 

In [64]:
display_all(joined_test.T)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,...,40988,40989,40990,40991,40992,40993,40994,40995,40996,40997,40998,40999,41000,41001,41002,41003,41004,41005,41006,41007,41008,41009,41010,41011,41012,41013,41014,41015,41016,41017,41018,41019,41020,41021,41022,41023,41024,41025,41026,41027,41028,41029,41030,41031,41032,41033,41034,41035,41036,41037,41038,41039,41040,41041,41042,41043,41044,41045,41046,41047,41048,41049,41050,41051,41052,41053,41054,41055,41056,41057,41058,41059,41060,41061,41062,41063,41064,41065,41066,41067,41068,41069,41070,41071,41072,41073,41074,41075,41076,41077,41078,41079,41080,41081,41082,41083,41084,41085,41086,41087
Id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,...,40989,40990,40991,40992,40993,40994,40995,40996,40997,40998,40999,41000,41001,41002,41003,41004,41005,41006,41007,41008,41009,41010,41011,41012,41013,41014,41015,41016,41017,41018,41019,41020,41021,41022,41023,41024,41025,41026,41027,41028,41029,41030,41031,41032,41033,41034,41035,41036,41037,41038,41039,41040,41041,41042,41043,41044,41045,41046,41047,41048,41049,41050,41051,41052,41053,41054,41055,41056,41057,41058,41059,41060,41061,41062,41063,41064,41065,41066,41067,41068,41069,41070,41071,41072,41073,41074,41075,41076,41077,41078,41079,41080,41081,41082,41083,41084,41085,41086,41087,41088
Store,1,3,7,8,9,10,11,12,13,14,15,16,19,20,21,22,23,24,25,27,29,30,31,32,33,35,36,38,39,40,41,42,43,45,46,47,48,49,50,51,52,53,56,58,61,62,63,64,66,67,68,69,70,71,72,73,74,75,76,77,79,80,81,82,83,84,86,89,90,91,92,93,94,98,99,100,101,102,105,107,108,109,110,111,112,113,115,117,118,119,120,122,124,126,127,128,129,130,131,132,...,980,983,984,985,986,988,989,991,992,994,997,998,1000,1003,1004,1005,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1019,1020,1022,1024,1025,1026,1027,1028,1031,1036,1037,1038,1039,1040,1041,1042,1044,1045,1047,1048,1049,1050,1051,1052,1053,1054,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1070,1071,1072,1073,1076,1077,1078,1079,1080,1083,1084,1086,1087,1088,1089,1091,1092,1094,1096,1097,1099,1100,1101,1102,1103,1104,1105,1106,1107,1109,1111,1112,1113,1114,1115
DayOfWeek,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,...,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
Date,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-17 00:00:00,2015-09-1

In [65]:
#fillna
for df in (joined,joined_test):
    df['CompetitionOpenSinceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
    df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
    df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1).astype(np.int32)

In [66]:
#adding more date relevant columns:
for df in (joined,joined_test):
    df["CompetitionOpenSince"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear, 
                                                     month=df.CompetitionOpenSinceMonth, day=15))
    df["CompetitionDaysOpen"] = df.Date.subtract(df.CompetitionOpenSince).dt.days

In [67]:
#removing too big and possibly wrong data:
for df in (joined,joined_test):
    df.loc[df.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
    df.loc[df.CompetitionOpenSinceYear<1990, "CompetitionDaysOpen"] = 0


In [68]:
#limiting it to 24
for df in (joined,joined_test):
    df["CompetitionMonthsOpen"] = df["CompetitionDaysOpen"]//30
    df.loc[df.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24
joined.CompetitionMonthsOpen.unique()

array([24,  3, 19,  9,  0, 16, 17,  7, 15, 22, 11, 13,  2, 23, 12,  4, 10,
        1, 14, 20,  8, 18,  6, 21,  5])

In [69]:
#NOTICE: THIS STRUCTURE:(apply() and lambda)
#When you pass the year and week of the year to Week(); it returns date(if you specify the day after.)
from isoweek import Week
for df in (joined,joined_test):
    df["Promo2Since"] = pd.to_datetime(df.apply(lambda x: Week(
        x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1))
    df["Promo2Days"] = df.Date.subtract(df["Promo2Since"]).dt.days

In [70]:
for df in (joined,joined_test):
    df.loc[df.Promo2Days<0, "Promo2Days"] = 0
    df.loc[df.Promo2SinceYear<1990, "Promo2Days"] = 0
    df["Promo2Weeks"] = df["Promo2Days"]//7
    df.loc[df.Promo2Weeks<0, "Promo2Weeks"] = 0
    df.loc[df.Promo2Weeks>25, "Promo2Weeks"] = 25
    df.Promo2Weeks.unique()

In [71]:
joined.to_pickle('joined')
joined_test.to_pickle('joined_test')

### durations:

In [72]:
#This func measures how may days passed from last event for that store:
def get_elapsed(fld, pre):
    day1 = np.timedelta64(1, 'D')
    last_date = np.datetime64()
    last_store = 0
    res = []

    for s,v,d in zip(df.Store.values,df[fld].values, df.Date.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: last_date = d
        res.append(((d-last_date).astype('timedelta64[D]') / day1)) #this basically gives day difference between d and last_date
    df[pre+fld] = res

In [73]:
display_all(joined_test[joined_test['Store']==1].T)

,0,856,1712,2568,3424,4280,5136,5992,6848,7704,8560,9416,10272,11128,11984,12840,13696,14552,15408,16264,17120,17976,18832,19688,20544,21400,22256,23112,23968,24824,25680,26536,27392,28248,29104,29960,30816,31672,32528,33384,34240,35096,35952,36808,37664,38520,39376,40232
Id,1,857,1713,2569,3425,4281,5137,5993,6849,7705,8561,9417,10273,11129,11985,12841,13697,14553,15409,16265,17121,17977,18833,19689,20545,21401,22257,23113,23969,24825,25681,26537,27393,28249,29105,29961,30817,31673,32529,33385,34241,35097,35953,36809,37665,38521,39377,40233
Store,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
DayOfWeek,4,3,2,1,7,6,5,4,3,2,1,7,6,5,4,3,2,1,7,6,5,4,3,2,1,7,6,5,4,3,2,1,7,6,5,4,3,2,1,7,6,5,4,3,2,1,7,6
Date,2015-09-17 00:00:00,2015-09-16 00:00:00,2015-09-15 00:00:00,2015-09-14 00:00:00,2015-09-13 00:00:00,2015-09-12 00:00:00,2015-09-11 00:00:00,2015-09-10 00:00:00,2015-09-09 00:00:00,2015-09-08 00:00:00,2015-09-07 00:00:00,2015-09-06 00:00:00,2015-09-05 00:00:00,2015-09-04 00:00:00,2015-09-03 00:00:00,2015-09-02 00:00:00,2015-09-01 00:00:00,2015-08-31 00:00:00,2015-08-30 00:00:00,2015-08-29 00:00:00,2015-08-28 00:00:00,2015-08-27 00:00:00,2015-08-26 00:00:00,2015-08-25 00:00:00,2015-08-24 00:00:00,2015-08-23 00:00:00,2015-08-22 00:00:00,2015-08-21 00:00:00,2015-08-20 00:00:00,2015-08-19 00:00:00,2015-08-18 00:00:00,2015-08-17 00:00:00,2015-08-16 00:00:00,2015-08-15 00:00:00,2015-08-14 00:00:00,2015-08-13 00:00:00,2015-08-12 00:00:00,2015-08-11 00:00:00,2015-08-10 00:00:00,2015-08-09 00:00:00,2015-08-08 00:00:00,2015-08-07 00:00:00,2015-08-06 00:00:00,2015-08-05 00:00:00,2015-08-04 00:00:00,2015-08-03 00:00:00,2015-08-02 00:00:00,2015-08-01 00:00:00
Open,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1
Promo,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0
StateHoliday,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
SchoolHoliday,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Year,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015
Month,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8


In [74]:
columns = ["Date", "Store", "Promo", "StateHoliday", "SchoolHoliday"]

In [75]:
#df = train[columns]
df = train[columns].append(test[columns])

In [90]:
fld = 'SchoolHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [79]:
fld = 'StateHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [88]:
fld = 'Promo'
df = df.sort_values(['Store', 'Date'])
get_elapsed(fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(fld, 'Before')

In [81]:
df = df.set_index("Date")

In [82]:
columns = ['SchoolHoliday', 'StateHoliday', 'Promo']

In [83]:
#filling all the missing values from the columns we've just made.
for o in ['Before', 'After']:
    for p in columns:
        a = o+p
        df[a] = df[a].fillna(0).astype(int)

In [84]:
#counting the number of events of interest for the last and next weeks.
bwd = df[['Store']+columns].sort_index().groupby("Store").rolling(7, min_periods=1).sum()

fwd = df[['Store']+columns].sort_index(ascending=False).groupby("Store").rolling(7, min_periods=1).sum()
                                      

In [91]:
display_all(df.T)

Date,2013-01-01,2013-01-02,2013-01-03,2013-01-04,2013-01-05,2013-01-06,2013-01-07,2013-01-08,2013-01-09,2013-01-10,2013-01-11,2013-01-12,2013-01-13,2013-01-14,2013-01-15,2013-01-16,2013-01-17,2013-01-18,2013-01-19,2013-01-20,2013-01-21,2013-01-22,2013-01-23,2013-01-24,2013-01-25,2013-01-26,2013-01-27,2013-01-28,2013-01-29,2013-01-30,2013-01-31,2013-02-01,2013-02-02,2013-02-03,2013-02-04,2013-02-05,2013-02-06,2013-02-07,2013-02-08,2013-02-09,2013-02-10,2013-02-11,2013-02-12,2013-02-13,2013-02-14,2013-02-15,2013-02-16,2013-02-17,2013-02-18,2013-02-19,2013-02-20,2013-02-21,2013-02-22,2013-02-23,2013-02-24,2013-02-25,2013-02-26,2013-02-27,2013-02-28,2013-03-01,2013-03-02,2013-03-03,2013-03-04,2013-03-05,2013-03-06,2013-03-07,2013-03-08,2013-03-09,2013-03-10,2013-03-11,2013-03-12,2013-03-13,2013-03-14,2013-03-15,2013-03-16,2013-03-17,2013-03-18,2013-03-19,2013-03-20,2013-03-21,2013-03-22,2013-03-23,2013-03-24,2013-03-25,2013-03-26,2013-03-27,2013-03-28,2013-03-29,2013-03-30,2013-03-31,2013-04-01,2013-04-02,2013-04-03,2013-04-04,2013-04-05,2013-04-06,2013-04-07,2013-04-08,2013-04-09,2013-04-10,...,2015-06-10,2015-06-11,2015-06-12,2015-06-13,2015-06-14,2015-06-15,2015-06-16,2015-06-17,2015-06-18,2015-06-19,2015-06-20,2015-06-21,2015-06-22,2015-06-23,2015-06-24,2015-06-25,2015-06-26,2015-06-27,2015-06-28,2015-06-29,2015-06-30,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,2015-07-11,2015-07-12,2015-07-13,2015-07-14,2015-07-15,2015-07-16,2015-07-17,2015-07-18,2015-07-19,2015-07-20,2015-07-21,2015-07-22,2015-07-23,2015-07-24,2015-07-25,2015-07-26,2015-07-27,2015-07-28,2015-07-29,2015-07-30,2015-07-31,2015-08-01,2015-08-02,2015-08-03,2015-08-04,2015-08-05,2015-08-06,2015-08-07,2015-08-08,2015-08-09,2015-08-10,2015-08-11,2015-08-12,2015-08-13,2015-08-14,2015-08-15,2015-08-16,2015-08-17,2015-08-18,2015-08-19,2015-08-20,2015-08-21,2015-08-22,2015-08-23,2015-08-24,2015-08-25,2015-08-26,2015-08-27,2015-08-28,2015-08-29,2015-08-30,2015-08-31,2015-09-01,2015-09-02,2015-09-03,2015-09-04,2015-09-05,2015-09-06,2015-09-07,2015-09-08,2015-09-09,2015-09-10,2015-09-11,2015-09-12,2015-09-13,2015-09-14,2015-09-15,2015-09-16,2015-09-17
Store,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115
Promo,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,1,1
StateHoliday,a,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,b,0,0,b,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
SchoolHoliday,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [95]:
display_all(bwd.T)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,...,1058197,1058198,1058199,1058200,1058201,1058202,1058203,1058204,1058205,1058206,1058207,1058208,1058209,1058210,1058211,1058212,1058213,1058214,1058215,1058216,1058217,1058218,1058219,1058220,1058221,1058222,1058223,1058224,1058225,1058226,1058227,1058228,1058229,1058230,1058231,1058232,1058233,1058234,1058235,1058236,1058237,1058238,1058239,1058240,1058241,1058242,1058243,1058244,1058245,1058246,1058247,1058248,1058249,1058250,1058251,1058252,1058253,1058254,1058255,1058256,1058257,1058258,1058259,1058260,1058261,1058262,1058263,1058264,1058265,1058266,1058267,1058268,1058269,1058270,1058271,1058272,1058273,1058274,1058275,1058276,1058277,1058278,1058279,1058280,1058281,1058282,1058283,1058284,1058285,1058286,1058287,1058288,1058289,1058290,1058291,1058292,1058293,1058294,1058295,1058296
Store,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115,1115
Date,2013-01-01 00:00:00,2013-01-02 00:00:00,2013-01-03 00:00:00,2013-01-04 00:00:00,2013-01-05 00:00:00,2013-01-06 00:00:00,2013-01-07 00:00:00,2013-01-08 00:00:00,2013-01-09 00:00:00,2013-01-10 00:00:00,2013-01-11 00:00:00,2013-01-12 00:00:00,2013-01-13 00:00:00,2013-01-14 00:00:00,2013-01-15 00:00:00,2013-01-16 00:00:00,2013-01-17 00:00:00,2013-01-18 00:00:00,2013-01-19 00:00:00,2013-01-20 00:00:00,2013-01-21 00:00:00,2013-01-22 00:00:00,2013-01-23 00:00:00,2013-01-24 00:00:00,2013-01-25 00:00:00,2013-01-26 00:00:00,2013-01-27 00:00:00,2013-01-28 00:00:00,2013-01-29 00:00:00,2013-01-30 00:00:00,2013-01-31 00:00:00,2013-02-01 00:00:00,2013-02-02 00:00:00,2013-02-03 00:00:00,2013-02-04 00:00:00,2013-02-05 00:00:00,2013-02-06 00:00:00,2013-02-07 00:00:00,2013-02-08 00:00:00,2013-02-09 00:00:00,2013-02-10 00:00:00,2013-02-11 00:00:00,2013-02-12 00:00:00,2013-02-13 00:00:00,2013-02-14 00:00:00,2013-02-15 00:00:00,2013-02-16 00:00:00,2013-02-17 00:00:00,2013-02-18 00:00:00,2013-02-19 00:00:00,2013-02-20 00:00:00,2013-02-21 00:00:00,2013-02-22 00:00:00,2013-02-23 00:00:00,2013-02-24 00:00:00,2013-02-25 00:00:00,2013-02-26 00:00:00,2013-02-27 00:00:00,2013-02-28 00:00:00,2013-03-01 00:00:00,2013-03-02 00:00:00,2013-03-03 00:00:00,2013-03-04 00:00:00,2013-03-05 00:00:00,2013-03-06 00:00:00,2013-03-07 00:00:00,2013-03-08 00:00:00,2013-03-09 00:00:00,2013-03-10 00:00:00,2013-03-11 00:00:00,2013-03-12 00:00:00,2013-03-13 00:00:00,2013-03-14 00:00:00,2013-03-15 00:00:00,2013-03-16 00:00:00,2013-03-17 00:00:00,2013-03-18 00:00:00,2013-03-19 00:00:00,2013-03-20 00:00:00,2013-03-21 00:00:00,2013-03-22 00:00:00,2013-03-23 00:00:00,2013-03-24 00:00:00,2013-03-25 00:00:00,2013-03-26 00:00:00,2013-03-27 00:00:00,2013-03-28 00:00:00,2013-03-29 00:00:00,2013-03-30 00:00:00,2013-03-31 00:00:00,2013-04-01 00:00:00,2013-04-02 00:00:00,2013-04-03 00:00:00,2013-04-04 00:00:00,2013-04-05 00:00:00,2013-04-06 00:00:00,2013-04-07 00:00:00,2013-04-08 00:00:00,2013-04-09 00:00:00,2013-04-10 00:00:00,...,2015-06-10 00:00:00,2015-06-11 00:00:00,2015-06-12 00:00:00,2015-06-13 00:00:00,2015-06-14 00:00:00,2015-06-15 00:00:00,2015-06-16 00:00:00,2015-06-17 00:00:00,2015-06-18 00:00:00,2015-0

In [93]:
bwd.drop('Store',1,inplace=True)
bwd.reset_index(inplace=True)

In [94]:
fwd.drop('Store',1,inplace=True)
fwd.reset_index(inplace=True)

In [96]:
df.reset_index(inplace=True)

In [97]:
df = df.merge(bwd, 'left', ['Date', 'Store'], suffixes=['', '_bw'])
df = df.merge(fwd, 'left', ['Date', 'Store'], suffixes=['', '_fw'])

In [99]:
df.drop(columns,1,inplace=True)

In [100]:
df.head()

,Date,Store,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo,SchoolHoliday_bw,Promo_bw,SchoolHoliday_fw,Promo_fw
0,2013-01-01,1,0,0,0,0,0,-6,1.0,0.0,7.0,1.0
1,2013-01-02,1,0,0,0,0,0,-5,2.0,0.0,7.0,2.0
2,2013-01-03,1,0,0,0,0,0,-4,3.0,0.0,7.0,3.0
3,2013-01-04,1,0,0,0,0,0,-3,4.0,0.0,7.0,4.0
4,2013-01-05,1,0,0,0,0,0,-2,5.0,0.0,7.0,5.0


In [ ]:
df.to_pickle('df')

In [101]:
df["Date"] = pd.to_datetime(df.Date)

In [102]:
df.columns

Index(['Date', 'Store', 'AfterSchoolHoliday', 'BeforeSchoolHoliday',
       'AfterStateHoliday', 'BeforeStateHoliday', 'AfterPromo', 'BeforePromo',
       'SchoolHoliday_bw', 'Promo_bw', 'SchoolHoliday_fw', 'Promo_fw'],
      dtype='object')

In [6]:
joined = pd.read_pickle('joined')
joined_test = pd.read_pickle('joined_test')

In [103]:
joined = join_df(joined, df, ['Store', 'Date'])

In [104]:
joined_test = join_df(joined_test, df, ['Store', 'Date'])

In [105]:
joined = joined[joined.Sales!=0]

In [108]:
joined_test.head()

,index,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Year,...,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo,SchoolHoliday_bw,Promo_bw,SchoolHoliday_fw,Promo_fw
0,0,1,1,4,2015-09-17,1.0,1,0,0,2015,...,13,0,0,0,0,0,0.0,4.0,0.0,1.0
1,1,2,3,4,2015-09-17,1.0,1,0,0,2015,...,37,0,0,0,0,0,0.0,4.0,0.0,1.0
2,2,3,7,4,2015-09-17,1.0,1,0,0,2015,...,20,0,0,0,0,0,0.0,4.0,0.0,1.0
3,3,4,8,4,2015-09-17,1.0,1,0,0,2015,...,20,0,0,0,0,0,0.0,4.0,0.0,1.0
4,4,5,9,4,2015-09-17,1.0,1,0,0,2015,...,37,0,0,0,0,0,0.0,4.0,0.0,1.0


In [107]:
joined.reset_index(inplace=True)
joined_test.reset_index(inplace=True)

In [5]:
joined.to_pickle('train_clean')
joined_test.to_pickle('test_clean')

NameError: name 'joined' is not defined